In [1]:
import sys
print("python 版本检测",sys.version)

python 版本检测 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [3]:
#gpu检测
import torch
print(f"PyTorch version: {torch.__version__}")
!nvidia-smi
print("torch可使用的GPU数量是",torch.cuda.device_count())

PyTorch version: 2.0.0+cu117
Wed Jun 12 09:50:25 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:09.0 Off |                    0 |
| N/A   27C    P8     9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                           